In [2]:
!pip install transformers

In [3]:
!pip install sentencepiece

In [4]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import XLNetTokenizer, XLNetForSequenceClassification, XLNetConfig, AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
train_ori = pd.read_csv("/kaggle/input/jbnu-swuniv-ai/train_data.csv")
test_ori = pd.read_csv("/kaggle/input/jbnu-swuniv-ai/test_data.csv")
train_desc = pd.read_csv("/kaggle/input/book-cover-prompt/merged_desc.csv")
test_desc = pd.read_csv("/kaggle/input/book-cover-prompt/merged_test.csv")

In [7]:
display(train_ori.head())
display(test_ori.head())
display(train_desc.head())
display(test_desc.head())

,id,Filename,Title,label
0,0,1101903236.jpg,The Oz Family Kitchen: More Than 100 Simple an...,"Cookbooks, Food, Wine"
1,1,0804139857.jpg,Living with Intent: My Somewhat Messy Journey ...,Self Help
2,2,0765334798.jpg,Redshirts: A Novel with Three Codas,"Science Fiction, Fantasy"
3,3,0446310786.jpg,To Kill a Mockingbird,"Mystery, Thriller, Suspense"
4,4,1143002598.jpg,Canning and Preserving of Food Products with B...,"Cookbooks, Food, Wine"


,id,Filename,Title
0,0,0.jpg,Elementary and Middle School Mathematics: Teac...
1,1,1.jpg,"Making Color Sing, 25th Anniversary Edition: P..."
2,2,2.jpg,Nursing Fundamentals DeMYSTiFieD: A Self-Teach...
3,3,3.jpg,Allen and Greenough's New Latin Grammar (Dover...
4,4,4.jpg,The Encyclopedia of Fantasy


,image,prompt
0,000713326X.jpg,"a book cover of taking risks with watercolour,..."
1,0007853939.jpg,"a book cover with a dragon and scroll on it, t..."
2,0020112602.jpg,a picture of a book cover with a drawing of a ...
3,002079990X.jpg,a close up of a person writing on a piece of p...
4,0027457702.jpg,a picture of a book cover with a man reading a...


,File,Prompt
0,0.jpg,a close up of a book cover with a bunch of dif...
1,1.jpg,making color sing practical lessons in color a...
2,10.jpg,a book cover of the book tibet through the red...
3,100.jpg,"a book cover of computer hardware, software, a..."
4,1000.jpg,arafed image of a city with a river and a rive...


In [8]:
train = pd.merge(train_ori, train_desc, left_on = 'Filename', right_on='image', how='inner')
train = train[['id', 'Filename', 'Title', 'label', 'prompt']]

test = pd.merge(test_ori, test_desc, left_on = 'Filename', right_on='File', how='inner')
test = test[['id', 'Filename', 'Title', 'Prompt']]
test.columns = ['id', 'Filename', 'Title', 'prompt']

In [9]:
torch.manual_seed(42)

In [10]:
train, valid = train_test_split(train, test_size=0.1, random_state=42)

In [11]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)

max_length = 600 #입력 시퀀스 최대 길이 (조정 가능)
train_input_texts = (train["Title"] + ", "+ train["prompt"]).tolist()
valid_input_texts = (valid["Title"] + ", "+ valid["prompt"]).tolist()
test_input_texts = (test["Title"] + ", "+ test["prompt"]).tolist()

train_encoded_inputs = tokenizer.batch_encode_plus(
    train_input_texts,
    max_length=max_length,
    padding="longest",
    truncation=True,
    return_tensors="pt"
)
valid_encoded_inputs = tokenizer.batch_encode_plus(
    valid_input_texts,
    max_length=max_length,
    padding="longest",
    truncation=True,
    return_tensors="pt"
)

test_encoded_inputs = tokenizer.batch_encode_plus(
    test_input_texts,
    max_length=max_length,
    padding="longest",
    truncation=True,
    return_tensors="pt"
)

In [12]:
train_input_ids = train_encoded_inputs["input_ids"].to(device)
train_attention_mask = train_encoded_inputs["attention_mask"].to(device)

label_mapping = {label: i for i, label in enumerate(train["label"].unique())}
num_labels = len(label_mapping)
train_labels = train["label"].map(label_mapping).tolist()
train_labels = torch.tensor(train_labels).to(device)

valid_input_ids = valid_encoded_inputs["input_ids"].to(device)
valid_attention_mask = valid_encoded_inputs["attention_mask"].to(device)

test_input_ids = test_encoded_inputs["input_ids"].to(device)
test_attention_mask = test_encoded_inputs["attention_mask"].to(device)

In [13]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=num_labels).to(device)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.weight', 'sequence_summary.summary.bias', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [14]:
batch_size = 16

train_data = torch.utils.data.TensorDataset(train_input_ids, train_attention_mask, train_labels)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

valid_data = torch.utils.data.TensorDataset(valid_input_ids, valid_attention_mask)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, shuffle=True)

test_data = torch.utils.data.TensorDataset(test_input_ids, test_attention_mask)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [15]:
# 최적화 설정 (선택 사항)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
num_epochs = 16

In [ ]:
import tqdm

model.train()

from sklearn.metrics import f1_score

best_f1_score = 0.0
patience = 10
counter = 0

for epoch in tqdm.tqdm(range(num_epochs)):
    train_loss = 0.0
    predicted_labels = []
    true_labels = []

    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()

        outputs = model(input_ids.to(device), attention_mask=attention_mask.to(device), labels=labels.to(device))
        loss = outputs.loss
        train_loss += loss.item()

        loss.backward()
        optimizer.step()

        predicted = model(input_ids.to(device), attention_mask=attention_mask.to(device)).logits.argmax(dim=-1)
        predicted_labels.extend(predicted.tolist())
        true_labels.extend(labels.tolist())

    avg_train_loss = train_loss / len(train_loader)
    f1 = f1_score(true_labels, predicted_labels, average='macro')

    print(f"Epoch: {epoch+1}, Average Training Loss: {avg_train_loss}, F1 Score: {f1}")

    if f1 > best_f1_score:
        best_f1_score = f1
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Training stopped due to no improvement in F1 score.")
            break


In [ ]:
model.eval()  # 모델을 평가 모드로 설정

valid_preds = []

with torch.no_grad():
    for batch in tqdm.tqdm(valid_loader):
        input_ids, attention_mask = batch  # 레이블은 필요하지 않으므로 "_"로 무시

        # 모델에 입력 전달하여 예측 수행
        outputs = model(input_ids.to(device), attention_mask=attention_mask.to(device))
        logits = outputs.logits

        # 예측된 장르 구분
        preds = torch.argmax(logits, dim=1)
        valid_preds.extend(preds.cpu().numpy())

# 예측된 장르 결과를 원본 데이터프레임에 추가
valid['Predicted Genre'] = valid_preds

# 결과 출력
print(valid[['Title', 'Predicted Genre']])

In [ ]:
valid_predicted_genres = [list(label_mapping.keys())[list(label_mapping.values()).index(pred)] for pred in valid_preds]
valid['Predicted Genre'] = valid_predicted_genres
print(valid[['Title', 'Predicted Genre']])

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(valid['label'],  valid['Predicted Genre']))

In [ ]:
model.eval()  # 모델을 평가 모드로 설정

test_preds = []

with torch.no_grad():
    for batch in tqdm.tqdm(test_loader):
        input_ids, attention_mask = batch  # 레이블은 필요하지 않으므로 "_"로 무시

        # 모델에 입력 전달하여 예측 수행
        outputs = model(input_ids.to(device), attention_mask=attention_mask.to(device))
        logits = outputs.logits

        # 예측된 장르 구분
        preds = torch.argmax(logits, dim=1)
        test_preds.extend(preds.cpu().numpy())

# 예측된 장르 결과를 원본 데이터프레임에 추가
test['Predicted Genre'] = test_preds

# 결과 출력
print(test[['Title', 'Predicted Genre']])

In [ ]:
test_predicted_genres = [list(label_mapping.keys())[list(label_mapping.values()).index(pred)] for pred in test_preds]
test['Predicted Genre'] = test_predicted_genres

In [ ]:
test = pd.read_csv("/kaggle/input/jbnu-swuniv-ai/test_data.csv")
test['label'] = test_predicted_genres
print(test[['Title', 'label']])

In [ ]:
test.to_csv('/kaggle/working/submission_prompt_10.csv', index=False, columns=['id', 'label'])